Import

In [ ]:
#Import
import argparse
import os
import numpy as np
import math
import torchvision.transforms as transforms
from torchvision.utils import save_image
from torch.utils.data import DataLoader
from torchvision import datasets
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, random_split, Dataset
from random import randint
from torch.optim.lr_scheduler import StepLR
import random;
from skimage.metrics import structural_similarity as ssim
import warnings
warnings.filterwarnings('ignore')

Generator

In [ ]:

#######################################################
# Generator Network
#######################################################


class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()

        #Embeddings Layers
        self.label_emb = nn.Embedding(n_classes, latent_dim)

        # Initial size before upsampling
        self.init_size = img_size // 4  

        #Resize
        self.l1 = nn.Sequential(nn.Linear(latent_dim, 128 * self.init_size ** 2))

        #Convolution Blocks
        self.conv_blocks = nn.Sequential(
            nn.BatchNorm2d(128),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(128, 128, 3, stride=1, padding=1),
            nn.BatchNorm2d(128, 0.8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(128, 64, 3, stride=1, padding=1),
            nn.BatchNorm2d(64, 0.8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(64, channels, 3, stride=1, padding=1),
            nn.Tanh(),
        )

    def forward(self, noise, labels):

        #Multiply Noise by Labels
        gen_input = torch.mul(self.label_emb(labels), noise)

        #Resize
        out = self.l1(gen_input)
        out = out.view(out.shape[0], 128, self.init_size, self.init_size)

        #Convolution Blocks
        img = self.conv_blocks(out)

        #Output
        return img;

Plot Training

In [ ]:
#Path
torch_path = r'/hpc/group/viplab/zzz3/SNP_Segmentation/Files/Torch/AAR-Net/';
experiment = 'Original'


#Save Data
discriminator_loss = np.load(torch_path + 'Discriminator_Loss_' + experiment +'.npy');
generator_loss = np.load(torch_path + 'Generator_Loss_' + experiment +'.npy');
discriminator_accuracy = np.load(torch_path +  'Discriminator_Accuracy_' + experiment +'.npy');
SSIM_results = np.load(torch_path +  'SSIM_Results_' + experiment +'.npy' );


#Epoch List
n_epochs = 400;
epochs = np.array(range(1,n_epochs+1))

#Plot Loss
plt.figure(figsize =(10,10));
plt.plot(epochs, discriminator_loss, label='Discriminator Loss');
plt.plot(epochs, generator_loss, label='Generator Loss');
plt.title('Discriminator vs. Generator Loss')
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend(loc='upper right', frameon=True);
plt.show();



#Plot Discriminator Accuracy 
plt.figure(figsize =(10,10));
plt.plot(epochs, discriminator_accuracy );
plt.title('Discriminator Accuracy')
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.show();


#Plot SSIM
plt.figure(figsize =(10,10));
plt.plot(epochs, SSIM_results[:,0], label='Subbasal SSIM');
plt.plot(epochs, SSIM_results[:,1], label='Epithelium SSIM');
plt.plot(epochs, SSIM_results[:,2], label='Stroma SSIM');
plt.title('Structural Similarity Index')
plt.xlabel("Epochs")
plt.ylabel("SSIM")
plt.legend(loc='upper right', frameon=True);
plt.show();

Load Model

In [ ]:
#Hyper Parameters
latent_dim = 400;
n_classes = 3;
img_size = 192;
channels = 1;


#Cast to Cuda
cuda = True if torch.cuda.is_available() else False
if cuda:
    FloatTensor = torch.cuda.FloatTensor
    LongTensor = torch.cuda.LongTensor
else:
    FloatTensor = torch.FloatTensor
    LongTensor = torch.LongTensor

#Attempt to use GPU instead of CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu');

#Load Generator
generator = Generator();
generator.load_state_dict(torch.load(torch_path +  'Generator_' + experiment + '.pth', map_location=device));
generator.to(device=device)

#Set the Network to Eval Mode for Prediction
generator.eval();

Plot Example Images

In [ ]:

#layer = 0; #SNP
#layer = 1; #Epithelium
layer = 2; #Stroma



# Get Sample Noise
n_row  =3;
z = Variable(FloatTensor(np.random.normal(0, 1, (n_row ** 2, latent_dim))))

# Get Labels
labels_fake = layer *np.ones((1,9)).squeeze();
labels_fake = Variable(LongTensor(labels_fake))


#Generate Images from Sample Noise and Labels
fake_imgs =  generator(z, labels_fake)
fake_imgs = fake_imgs.cpu().detach().numpy().squeeze();

#Iterate
for i in range(0,9):

    plt.figure;
    plt.imshow(fake_imgs[i],cmap='gray');
    plt.show();